## Nuestra bitácora para hacer filogenias hk-tr

Programas requeridos 
-HAMMER
-MUSCLE
-FASTTREE

Archivos requeridos
-GENOMAS (filtraados por contigS
-Base de datos: MiBIG, Pfam-A.hmm
-antiSMASH (data bajada de RAST archivo .gbk)
-BiGFAM (BGCs completos)

Toda la información de este trabajo se encuentra depositada en el Github del laboratorio: IndustrialGenomicsLab

## Paso 1. Descarga de secuencias de aminoacidos desde RAST

Copiar la lista de jobs de la cuenta de RAST. Se recuperaron solo los trabajos correspondientes a genomas de Cuatro Cienegas. La lista se paso a un archivo nano. Solo se dejo el numero de trabajo, ID, y nombre separado por tabulaciones. 
El archivo se guardo bajo en nombre "RAST_id" en la carpeta Data/GENOMES/genomas. Para instalar myrast se descargo el programa con la instruccion

In [ ]:
docker pull nselem/myrast

La terminal de myrast se abrio con la instrucción

In [ ]:
docker run -i -t -v $(pwd):/home nselem/myrast /bin/bash

Las secuencias de aminoacidos se descargaron pocisionado en la carpeta de destino con la instruccion

In [ ]:
cut -d ' ' -f 1 rastID | while read line; do svr_retrieve_RAST_job industrial <password> $line amino_acid > CA_domain/$line.faa ; done

## Paso 2. Buscando dominios selectos en genomas de Cuatro Cienegas

Con la funcion grep se buscaron los dominios catalíticos (CA) de interes del archivo Dominios en el archivo Pfam-A.hmm

In [5]:
%%bash 
cut -f 1 CA_domain.txt | while read line; do grep $line ../../sources/Pfam-A.hmm | cut -d ' ' -f 4 ; done > CA_new

In [6]:
%%bash
head CA_new

PF02518.29
PF13581.9
PF13589.9
PF13749.9
PF14501.9


## Paso 3. Search of CA domains in Cuatro Cienegas and Abyssal genomes
Actual accesion number of CA domains were recovered from Pfam-A.hmm file using CAdomain.txt file

In [8]:
%%bash
ls CA_domain/*faa | while read line; do cat CA_new | hmmfetch -f ../../sources/Pfam-A.hmm - |hmmsearch - $line > $line.out; done

In [9]:
%%bash
ls CA_domain/*.out | while read line; do perl ../../sources/pFAM-parser.pl $line; done

hello world
fig|6666666.737887.peg.6243
fig|6666666.737887.peg.1191
fig|6666666.737887.peg.6876
fig|6666666.737887.peg.4368
fig|6666666.737887.peg.6119
fig|6666666.737887.peg.4326
fig|6666666.737887.peg.6523
fig|6666666.737887.peg.3056
fig|6666666.737887.peg.5238
fig|6666666.737887.peg.4946
fig|6666666.737887.peg.2796
fig|6666666.737887.peg.1525
fig|6666666.737887.peg.5337
fig|6666666.737887.peg.6232
fig|6666666.737887.peg.5946
fig|6666666.737887.peg.3100
fig|6666666.737887.peg.5000
fig|6666666.737887.peg.1837
fig|6666666.737887.peg.3028
fig|6666666.737887.peg.3745
fig|6666666.737887.peg.636
fig|6666666.737887.peg.7300
fig|6666666.737887.peg.1797
fig|6666666.737887.peg.954
fig|6666666.737887.peg.6099
fig|6666666.737887.peg.7296
fig|6666666.737887.peg.4058
fig|6666666.737887.peg.224
fig|6666666.737887.peg.2057
fig|6666666.737887.peg.6081
fig|6666666.737887.peg.6404
fig|6666666.737887.peg.5611
fig|6666666.737887.peg.2900
fig|6666666.737887.peg.977
fig|6666666.737887.peg.966
fig|6666666.7

In [1]:
%%bash
cat CA_domain/*out-PF* | sort | uniq > CA_domain/CA_lab.txt

a la lista con secuencias de dominio CA se le van a filtrar las secuencias que corresponden a clusters completos usando el archivo gen_ids_complete

In [13]:
%%bash
cut CA_domain/gen_ids_complete.tsv -f1 | while read line; do grep -w $line CA_domain/CA_lab.txt >> CA_domain/CA_BGCfilter.id; done

CalledProcessError: Command 'b'cut CA_domain/gen_ids_complete.tsv -f1 | while read line; do grep -w $line CA_domain/CA_lab.txt >> CA_BGCfilter.id; done\n'' returned non-zero exit status 1.

Con la lista de CA filtrados se recuperaron las secuencias de A.A en archivos .fasta

In [14]:
%%bash
cat CA_domain/CA_BGCfilter.id | while read line 
do 
echo '>'$line >> CA_domain/CA_BGCfilter.fasta
grep -A 1000 -w -h $line CA_domain/*faa | sed -n -e '1,/>/ {/>/ !{'p''}} >> CA_domain/CA_BGCfilter.fasta
done

## Paso 4. Search of CA domains in MIBiG clusters


Proteins with CA domain in MIBiG clusters were search

In [2]:
%%bash
ls MIBIG/*fasta |while read line; do cat CA_new | hmmfetch -f ../../sources/Pfam-A.hmm - |hmmsearch - $line > $line.out; done

ID of proteins were recovered with script pFAM-parser.pl

In [3]:
%%bash
ls MIBIG/*out | while read line; do perl ../../sources/pFAM-parser.pl $line; done

hello world
BGC0002007_WP_091320338.1
BGC0000043_ACV42481.1
BGC0000080_ABK32300.1
BGC0001417_APZ78674.1
BGC0001417_orf1
BGC0000014_ABK32275.1
BGC0002026_QDA77064.1
BGC0000997_CAL80818.1
BGC0000080_ABK32285.1
BGC0001558_WP_018891765.1
BGC0000023_CAO98841.1
BGC0000527_CAB60253.1
BGC0000223_AEE65496.1
BGC0000080_ABK32283.1
BGC0001961_QCE20597.1
BGC0000014_ABK32253.1
BGC0000133_AFI57016.1
BGC0000842_AGU42431.1
BGC0002025_QDA77040.1
BGC0000845_EDY47142.1
BGC0000014_ABK32251.1
BGC0000014_ABK32279.1
BGC0001288_BAQ25485.1
BGC0000445_CAJ34390.1
BGC0001469_RAT94095.1
BGC0000833_AAO47225.2
BGC0000188_ABI75118.1
BGC0000928_ABI75118.1
BGC0001093_AAY32961.1
BGC0000834_AFI47646.1
BGC0002026_QDA77063.1
BGC0000832_AAN65247.1
BGC0000833_AAO47226.2
BGC0000515_ABO65654.1
BGC0000983_EAW09112.1
BGC0001964_AYM52840.1
BGC0000084_AFL48547.1
BGC0000545_CAB93670.1
BGC0000171_CCC21117.1
BGC0000260_ABB69090.1
BGC0001380_ALI93253.1
BGC0001765_AMD57702.1
BGC0000668_AHI58828.1
BGC0001686_CDE97357.1
BGC0001387_AMP4659

IDs were corrected to match with actual IDs (el script de Nelly borra los guiones bajos por lo que los siguientes comandos organizan la información)

In [4]:
%%bash
perl -p -i -e 's/\n/_/g' MIBIG/*PF*

In [5]:
%%bash
perl -p -i -e 's/_BGC0/\nBGC0/g' MIBIG/*PF*

In [6]:
%%bash
perl -p -i -e 's/_$/\n/' MIBIG/*PF*

In [7]:
%%bash
sed -i 1d MIBIG/*PF*

Vamos a agrupar todos los IDs que tienen dominio CA y los duplicados se eliminaron

In [9]:
%%bash
cat MIBIG/*out-PF* | sort | uniq > MIBIG/CA_MIBIG.txt

CA-proteins sequences were recovered using MIBIG/MIBiG_CA.id file

In [12]:
%%bash
cat MIBIG/CA_MIBIG.txt | while read id
    do
    echo '>'$id >> MIBIG/CA_MIBIG.fasta
    grep -A 1000 -w -h $id MIBIG/mibig_prot_seqs_ids.fasta | sed -n -e '1,/>/ {/>/ !{'p''}} >> MIBIG/CA_MIBIG.fasta
done

## Paso 5. Histidine Kinase match
CA-proteins of MIBIG and genomic sequences were combine in one file

In [15]:
%%bash
cat CA_domain/CA_BGCfilter.fasta MIBIG/CA_MIBIG.fasta > HK/CA_all.fasta

In [16]:
%%bash 
cut -f 1 HK/His-KA-Pfam.txt | while read line; do grep $line ../../sources/Pfam-A.hmm | cut -d ' ' -f 4 ; done > HK/His-ka_new

In [17]:
%%bash
cat HK/His-ka_new | hmmfetch -f ../../sources/Pfam-A.hmm - |hmmsearch - HK/CA_all.fasta > HK/CA_all.fasta.out

In [18]:
%%bash
perl ../../sources/pFAM-parser.pl HK/CA_all.fasta.out

hello world
BGC0001755_AWW87432.1
BGC0000845_EDY47142.1
BGC0001558_WP_018891765.1
BGC0000014_ABK32275.1
BGC0000080_ABK32300.1
fig|6666666.563200.peg.6170
BGC0000043_ACV42481.1
BGC0002007_WP_091320338.1
BGC0000310_AAD21212.2
fig|6666666.563200.peg.2791
fig|6666666.867958.peg.1682
fig|6666666.867961.peg.285
BGC0001961_QCE20597.1
fig|6666666.590940.peg.3184
fig|6666666.883177.peg.6211
fig|6666666.540615.peg.894
fig|6666666.871043.peg.5674
fig|6666666.590942.peg.3043
BGC0002026_QDA77064.1
fig|6666666.540227.peg.7220
fig|6666666.540227.peg.7277
fig|6666666.590942.peg.6048
BGC0000445_CAJ34390.1
BGC0000983_EAW09112.1
fig|56099.17.peg.4158
BGC0000842_AGU42431.1
BGC0000080_ABK32285.1
BGC0001288_BAQ25485.1
fig|6666666.737888.peg.5803
BGC0000014_ABK32253.1
fig|6666666.540227.peg.6214
fig|6666666.867962.peg.6839
fig|6666666.540227.peg.6673
BGC0000997_CAL80818.1
fig|56099.17.peg.6266
BGC0000260_ABB69090.1
fig|6666666.590942.peg.6040
fig|6666666.871043.peg.4045
BGC0000035_AAS79470.1
fig|6666666.5406

In [19]:
%%bash
perl -p -i -e 's/\n/_/g' HK/*PF*

In [20]:
%%bash
perl -p -i -e 's/_BGC0/\nBGC0/g' HK/*PF*

In [21]:
%%bash
perl -p -i -e 's/_fig/\nfig/g' HK/*PF*

In [22]:
%%bash
perl -p -i -e 's/_$/\n/' HK/*PF*

In [23]:
%%bash
sed -i 1d HK/*PF*

In [24]:
%%bash
ls HK/*out-PF* | while read list
    do
    cat $list | while read id
        do
        echo '>'$id >> $list.fasta
        grep -A 1000 -w -h $id HK/CA_all.fasta | sed -n -e '1,/>/ {/>/ !{'p''}} >> $list.fasta
    done
done

Ahora lo repetimos para las HK tipo I

In [25]:
%%bash
cat Class-I_HK | cut -f 1 | while read list
    do
    grep '>' HK/*$list*fasta >> HK/Class-I_HK
    done
cat HK/Class-I_HK | sort | uniq > HK/Class-I_HK.id

grep: HK/*PF19191*fasta: No such file or directory


In [26]:
%%bash
cat HK/Class-I_HK.id | while read id
    do
    echo $id >> HK/Class-I_HK.fasta
    grep -A 1000 -w -h $id HK/CA_all.fasta | sed -n -e '1,/>/ {/>/ !{'p''}} >> HK/Class-I_HK.fasta
    done

## Paso 6. Alineamientos con MUSCLE

In [ ]:
%%bash
ls HK/*PF*fasta | while read line
    do
    muscle -in $line -out $line.afa 
    mv $line.afa alignments/
    done

In [ ]:
%%bash
muscle -in HK/Class-I_HK.fasta -out alignments/Class-I_HK.afa 

## Paso 7. arboles con fasttree

In [27]:
%%bash
ls alignments/*afa | while read line
    do
    FastTree < $line > $line.nwk
    mv $line.nwk trees/
    done

FastTree Version 2.1.11 Double precision (No SSE3)
Alignment: standard input
Amino acid distances: BLOSUM45 Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Jones-Taylor-Thorton, CAT approximation with 20 rate categories
Initial topology in 0.23 seconds
Refining topology: 26 rounds ME-NNIs, 2 rounds ME-SPRs, 13 rounds ML-NNIs
      0.22 seconds: ME NNI round 1 of 26, 1 of 95 splits
      0.82 seconds: SPR round   1 of   2, 101 of 192 nodes
      1.35 seconds: ME NNI round 9 of 26, 1 of 95 splits
      1.88 seconds: SPR round   2 of   2, 101 of 192 nodes
      2.41 seconds: ME NNI round 17 of 26, 1 of 95 splits
Total branch-length 60.979 after 2.48 sec
      4.00 seconds: ML NNI round 1 of 13, 1 of 95 splits
ML-NNI round 1: LogLk = -116964.268 NNIs 14 max delta 6.83 Time 9.65
      9.71 seconds: Site likelihoods with rate category 1 of 20
      9.83 seconds: Site likelihoods wit